# Homework: Phrasal Chunking

This is your documentation for the chunker homework.

In [1]:
import perc
import default
import sys
from collections import defaultdict

feat_vec = {}
tagset = []
train_data = []

def get_tag(labeled_list, i):
    return labeled_list[i].split()[2]

def add_weight(weights, features, tag, number):
    for f in features:
        key = (f, tag)
        if key not in weights:
            weights[key] = 0
        weights[key] += number

def perc_train(train_data, tagset, numepochs):
    #print(train_data[0])
    print(len(train_data))
    assert len(train_data[0]) == 2
    labeled_list, feat_list = train_data[0][0], train_data[0][1]
    weights = {}
    for x in range(3):
        for i in range(len(train_data)):
            if i % 100 == 1:
                print(i)
            lebeled_list, feat_list = train_data[i]
            weights = perc_train_single(labeled_list, feat_list, tagset, weights)
    # print(weights)
    return weights

def perc_train_single(labeled_list, feat_list, tagset, weights):
    #print(feat_list)
    #print(labeled_list)
    #print(labeled_list)
    out = perc.perc_test(weights, labeled_list, feat_list, tagset, tagset[0])
    assert len(out) == len(labeled_list)
    feat_index = 0
    for i in range(len(labeled_list)):
        right_tag = get_tag(labeled_list, i)
        tag = out[i]
        feat_index, feats = perc.feats_for_word(feat_index, feat_list)
        if tag != right_tag:
            add_weight(weights, feats, right_tag, +1)
            add_weight(weights, feats, tag, -1)
            
        # for each tag fix the weights of all features pertaining to that tag
        
        
    # print(out)
    #print(out)
    # feat_vec = defaultdict(int)
    # insert your code here
    # please limit the number of iterations of training to n iterations
    printed_weights = {}
    for key, value in weights.items():
        if value != 0:
            printed_weights[key] = value
    #print(printed_weights)
    
    return weights
    return feat_vec

tagset = perc.read_tagset("data/tagset.txt")
print("reading data ...", file=sys.stderr)
train_data = perc.read_labeled_data("data/train.txt.gz", "data/train.feats.gz", verbose=False)
print("done.", file=sys.stderr)
feat_vec = perc_train(train_data, tagset, 5)
perc.perc_write_to_file(feat_vec, "default.model")
print("wrote model to disk")

reading data ...
done.


8936
1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
3101
3201
3301
3401
3501
3601
3701
3801
3901
4001
4101
4201
4301
4401
4501
4601
4701
4801
4901
5001
5101
5201
5301
5401
5501
5601
5701
5801
5901
6001
6101
6201
6301
6401
6501
6601
6701
6801
6901
7001
7101
7201
7301
7401
7501
7601
7701
7801
7901
8001
8101
8201
8301
8401
8501
8601
8701
8801
8901
1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
3101
3201
3301
3401
3501
3601
3701
3801
3901
4001
4101
4201
4301
4401
4501
4601
4701
4801
4901
5001
5101
5201
5301
5401
5501
5601
5701
5801
5901
6001
6101
6201
6301
6401
6501
6601
6701
6801
6901
7001
7101
7201
7301
7401
7501
7601
7701
7801
7901
8001
8101
8201
8301
8401
8501
8601
8701
8801
8901
1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2

In [2]:
%%capture --no-stderr output
print("reading test data ...", file=sys.stderr)
test_data = perc.read_labeled_data("data/dev.txt", "data/dev.feats", verbose=False)
print("done.", file=sys.stderr)
feat_vec = perc.perc_read_from_file("default.model")
perc.perc_testall(feat_vec, test_data, tagset)

reading test data ...
done.


In [3]:
import score_chunks
boundary = "-X-" # something to use as boundary between sentences
outside = "O" # tag used to mark the outside of any chunk
conlleval = False # use conlleval (should be False for most use cases)
numfeats = 2 # number of columns to consider as features, typically "word POStag"
(test, _) = score_chunks.readTestFile(str(output), boundary, outside, conlleval, numfeats)
with open("data/reference500.txt") as f:
    (reference, _) = score_chunks.readTestFile(f.read(), boundary, outside, conlleval, numfeats)
print("Score: %.2f" % score_chunks.corpus_fmeasure(reference, test, False))

processed 500 sentences with 10375 tokens and 5783 phrases; found phrases: 8706; correct phrases: 847
             ADJP: precision:   0.32%; recall:   1.01%; F1:   0.49; found:    312; correct:     99
             ADVP: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:    202
            CONJP: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:      5
             INTJ: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:      1
               NP: precision:   9.02%; recall:  14.67%; F1:  11.17; found:   4921; correct:   3026
               PP: precision:  14.83%; recall:  27.60%; F1:  19.30; found:   2272; correct:   1221
              PRT: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:     22
             SBAR: precision:   0.61%; recall:   0.93%; F1:   0.74; found:    164; correct:    107
               VP: precision:   6.17%; recall:   5.82%; F1:   5.99; found:   1037; correct:   1100
accurac